## import libraries

In [159]:
# !pip install jsonify
from jsonify import convert

In [1]:
import cv2 as cv
import numpy as np
import pytesseract as pt

import matplotlib.pyplot as plt

import fitz

import os
import numpy as np
from PIL import Image
from fpdf import FPDF
# from PyPDF2 import PdfReader

## image preprocessing

## add text to given location

In [187]:
from collections import defaultdict

class Preprocess():
    def __init__(self, img):
        self.img = img
        
    def pre_process(self):
        gray_image = cv.cvtColor(self.img, cv.COLOR_BGR2GRAY)

        ret,thresh = cv.threshold(gray_image, 0, 255, cv.THRESH_OTSU | cv.THRESH_BINARY_INV)

        #dilation
        kernel = cv.getStructuringElement(cv.MORPH_RECT, (14, 14))

        # Applying dilation on the threshold image
        dilated_image = cv.dilate(thresh, kernel, iterations = 1)

        return dilated_image


class Recognize():
    def __init__(self, final_img):
        self.final_img = final_img
        
    def rect(self, original_img):
        
        original_img_area = original_img.shape[0]*original_img.shape[1]
        
        outer_dict = defaultdict(dict)
        
        (contours, heirarchy) = cv.findContours(self.final_img, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
        sorted_contours_lines = sorted(contours, key = lambda ctr : cv.boundingRect(ctr)[0])
        
        test = []
        for line in sorted_contours_lines:
            x, y, w, h = cv.boundingRect(line)
            rect = (x, y, x+w, y+h)
            rect_area = (rect[2]-rect[0])*(rect[3]-rect[1])
            
            if cv.contourArea(line) > 1500 and rect_area != original_img_area:
                image = original_img[rect[1]:rect[3], rect[0]:rect[2]]
    
                text = pt.image_to_string(image, config=custom_config)
                
                inner_dict = {"text": text, "text_box": rect}
                
        outer_dict["page"]["text"] = inner_dict
                
#         page_info["page"] = test
                
        return outer_dict

In [188]:
custom_config = r'-l eng --oem 3 --psm 1'

image = cv.imread("test_file/test.jpeg")
preprocess = Preprocess(image)
pre_processed_image = preprocess.pre_process()

In [189]:
recognize = Recognize(pre_processed_image)
rect_lists = recognize.rect(image)
print(rect_lists["page"])

{'text': {'text': 'Pappadeaux Salad\n', 'text_box': (985, 329, 1259, 370)}}


In [171]:
import json

with open("sample.json", "w") as outfile:
    json.dump(rect_lists, outfile)